# Solve Wave Equation with PINN

## Defining the Wave Equation

The wave equation in 1D is given by:
$$
\frac{\partial^2u}{\partial t^2}=c^2\frac{\partial^2u}{\partial x^2}
$$

where $u$ is the wave function, $c$ is the wave speed and $x$ is the spatial coordinate, and $t$ is the time.

## Neural Network Setup
- Input: $(x,t)$
- Output: $u$

## PDE Residual

Using automatic differentiation to compute the required derivatives:
$$
\text{Residual} = \frac{\partial^2u}{\partial t^2}-c^2\frac{\partial^2u}{\partial x^2}
$$

## Loss Function

- **Data Loss** : Mean squared error between predicted $u$ and known data points.
- **PDE Residual Loss**: Mean squared error of the PDE residual.

## Training

Train the neural network to minimize the combined loss function using gradient descent.